In [2]:
#imports
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

In [3]:
#import the fnirs data
import os 
sampling_rate = 3.8147
fnirs_data = sio.loadmat('./data/data.mat')
fnirs_data = fnirs_data['structList']
len(fnirs_data)

23

In [4]:
fnirs_data[0][0][0][0][1].shape

(11521, 178)

In [5]:
data = []
for i in range(len(fnirs_data)):
    data.append(fnirs_data[i][0][0][0][1])
len(data)

23

In [6]:
demographics = [
    'P07', 'P10', 'P11', 'P13', 'P14', 'P16', 'P17', 'P18', 'P22', 'P23', 
    'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 
    'P34', 'P35', 'P36'
]

#load the lsl files
timings = []
for subject in demographics:
    path = os.path.join("data", "lsl", subject+"_lsl.tri")
    timing = pd.read_csv(path, sep=";", header=None)
    timings.append(timing)
print(timings[0].shape)
print(timings[0].head())


(362, 3)
                            0     1   2
0  2024-05-27T16:54:02.533199  1583  30
1  2024-05-27T16:54:16.907441  1638  40
2  2024-05-27T16:54:26.576106  1675  90
3  2024-05-27T16:54:35.683827  1709  20
4  2024-05-27T16:54:48.103673  1757  50


In [7]:
print(len(timings))
print(len(data))
print(len(demographics))

23
23
23


In [8]:
def get_onset(timing, sampling_rate):
    timing = timing / sampling_rate
    return timing

In [11]:
#sanity check data

P07 = data[0]
onsets = timings[0]
print(P07.shape)
sample = onsets.iloc[0, 1]
fnirs_sample = P07[sample]
fnirs_sample.shape

(11521, 178)


(178,)